In [1]:
from pathlib import Path
from evaluation_utils import get_document_text
import pandas as pd
import numpy as np
import json

dcc_dir = Path('data') / 'EMCDutchClinicalCorpus'
data_dir = Path('data')
result_dir = Path('results')
bilstm_result_file = result_dir / 'bilstm_predictions_cv.csv.gz'
robbert_result_file = result_dir / 'robbert_v2_dutch_base_3_32_512_predictions.csv.gz'
medroberta_file = result_dir / 'MedRoBERTa_3_32_512_predictions.csv.gz'
merged_result_file = result_dir / 'merged_results.csv.gz'
annotation_file = data_dir / 'emc-dcc_ann.json'

# Load results
results = pd.read_csv(merged_result_file)
results.head()

,entity_id,category,label,bilstm,bilstm_cv,rule_based,robbert_v2_dutch_base_2_64_512,robbert_v2_dutch_base_3_64_512,robbert_v2_dutch_base_3_32_512,robbert_v2_dutch_base_3_32_32,robbert_v2_dutch_base_3_32_64,MedRoBERTa_2_64_512,MedRoBERTa_3_64_512,MedRoBERTa_3_32_512,robbert_v2_dutch_base_DAPT_2_64_512,robbert_v2_dutch_base_DAPT_3_64_512,robbert_v2_dutch_base_DAPT_3_32_512,robbert_v2_dutch_base_10_128_512_zeroShot,robbert_v2_dutch_base_DAPT_10_128_512_zeroShot,MedRoBERTa_10_128_512_zeroShot
0,DL1111_32_46,DL,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated
1,DL1111_272_280,DL,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated
2,DL1111_363_377,DL,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated
3,DL1112_22_28,DL,negated,negated,negated,negated,negated,negated,negated,negated,negated,negated,negated,negated,negated,negated,negated,negated,negated,negated
4,DL1113_59_67,DL,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated


# RobBERT

In [2]:
colname = 'robbert_v2_dutch_base_3_32_512' # MedRoBERTa_3_32_512

file = robbert_result_file 
results = pd.read_csv(file)
results.drop_duplicates().dropna().to_csv('results/robbert_v2_dutch_base_3_32_512_predictions.csv.gz')

file = result_dir / 'robbert_v2_dutch_base_DAPT_3_32_512_predictions.csv.gz' 
results = pd.read_csv(file)
results.drop_duplicates().dropna().to_csv('results/robbert_v2_dutch_base_DAPT_3_32_512_predictions.csv.gz')

file =  result_dir / 'MedRoBERTa_3_32_512_predictions.csv.gz'
results = pd.read_csv(file)
results.drop_duplicates().dropna().to_csv('results/MedRoBERTa_3_32_512_predictions.csv.gz')

print(results.shape[0], results.isna().sum())
results.head()

12431 Unnamed: 0             0
entity_id              0
category               0
label                  0
MedRoBERTa_3_32_512    0
dtype: int64


,Unnamed: 0,entity_id,category,label,MedRoBERTa_3_32_512
0,0,DL1111_272_280,DL,not negated,not negated
1,1,DL1111_32_46,DL,not negated,not negated
2,2,DL1111_363_377,DL,not negated,not negated
3,3,DL1112_22_28,DL,negated,negated
4,4,DL1113_141_155,DL,not negated,not negated


## Overall

In [7]:
results.category.unique()

array(['DL', 'GP', 'RD', 'SP'], dtype=object)

In [12]:
scores = []

FN = results[(results.label=='negated') & (results.robbert_v2_dutch_base_3_32_512=='not negated')].shape[0]
TN = results[(results.label=='not negated') & (results.robbert_v2_dutch_base_3_32_512=='not negated')].shape[0]
FP = results[(results.label=='not negated') & (results.robbert_v2_dutch_base_3_32_512=='negated')].shape[0]
TP = results[(results.label=='negated') & (results.robbert_v2_dutch_base_3_32_512=='negated')].shape[0]

ACC = (TP+TN)/(TP+TN+FP+FN)
SPEC = TN/(TN+FP)
SENS = TP/(TP+FN)
PPV = TP/(TP+FP)
NPV = TN/(TN+FN)

missing_perc = results.robbert_v2_dutch_base_3_32_512.isna().sum()/results.shape[0]

scores.append({'blocksize': 512, 'acc': ACC, 'spec': SPEC, 'sens': SENS, 'ppv': PPV, 'npv': NPV, 'miss':missing_perc})
scores_df = pd.DataFrame(scores)

In [13]:
scores_df

,blocksize,acc,spec,sens,ppv,npv,miss
0,512,0.984523,0.992074,0.939812,0.952434,0.989859,0.010421


Going from a block size of $512$ to a block-size of $32$ does not lead to a noticeable performance decrease, it does decrease the computation time by a factor of $10$. We have to note here that due to the current batch process we skip label tokens that are outside the block size. 

## False negatives

In [14]:
false_negatives = results[(results.label == 'negated') & (results.robbert_v2_dutch_base_3_32_512== 'not negated')]
false_negatives.head()
print(false_negatives.shape[0])

false_positives = results[(results.label == 'not negated') & (results.robbert_v2_dutch_base_3_32_512== 'negated')]
false_positives.head()
print(false_positives.shape[0])

109
85


In [229]:
# Show issue with a random record
random_entity = false_negatives.entity_id.tolist()[109]
text, start, end = get_document_text(random_entity, dcc_dir, results)

IndexError: list index out of range

In [198]:
# Show issue with a random record
random_entity = false_positives.entity_id.tolist()[85]
text, start, end = get_document_text(random_entity, dcc_dir, results)

IndexError: list index out of range

* Uncertainty splitsen in: “speculation” en “ambiguous”
* Annotation error splitsen in: “speculation”, “ambiguous” of toch “annotation_error”

In [230]:
error_types = {'false_negative': {'uncommon_negation': [0,4,11,45,49,74,82,88,89,92,96,
                                                        101,102,103,104,105,107], 
                                  'annotation_error': [10,13,20,21,27,36,39,52,54,65,66,67,78,93,108],                                  
                                  'speculation': [2,3,5,6,8,18,29,40,43,44,
                                                  47,50,53,55,58,59,60,61,68,80,81], 
                                  'ambiguous': [1,7,9,19,41,57,62,76],
                                  'minus': [12,15,16,17,22,23,24,25,26,
                                            28,30,31,33,34,37,38,42,63,83,84,85,86,106],
                                  'other': [14,32,35,46,48,51,56,73,77,90,94,95,97,98,99],
                                  'scope': [6,64,70,71,72,75,79,91],
                                  'punctuation': [100],
                                  'wrong_modality': [69,87]
                                 },
              'false_positive': {'annotation_error': [0,9,11,22,28,32,47,60,61,68,76,79,82],
                                 'negation_of_different_term': [1,4,5,6,7,14,15,18,19,
                                                                20,24,26,29,30,37,38,40,49,52,
                                                                55,62,63,64,65,66,69,70,71,72,
                                                                74,77,78],
                                 'speculation': [10,16,39,45,46,56,57,59],
                                 'ambiguous': [2,3,8,12,17,42,43,44,48,53,73],
                                 'punctuation': [35],
                                 'other': [23,25,27,31,33,34,41,50,51,54,58,75,80,81,83,84],
                                 'scope': [],                                                 
                                 'minus': [],
                                 'wrong_modality': [13,21,36,67]
              }}

In [231]:
tmp = pd.DataFrame(error_types)
tmp['false_negative'] = tmp['false_negative'].apply(lambda x: x if isinstance(x,list) else [])
tmp['false_positive'] = tmp['false_positive'].apply(lambda x: x if isinstance(x,list) else [])
tmp['fn_count'], tmp['fp_count'] = zip(*tmp.apply(lambda x: (len(x[0]), len(x[1])), axis=1))
tmp[['fn_perc','fp_perc']] = tmp[['fn_count', 'fp_count']]/tmp[['fn_count', 'fp_count']].sum(axis=0)

In [232]:
tmp[['fn_count', 'fp_count', 'fn_perc', 'fp_perc']].round(3)

,fn_count,fp_count,fn_perc,fp_perc
uncommon_negation,17,0,0.155,0.000
annotation_error,15,13,0.136,0.153
speculation,21,8,0.191,0.094
ambiguous,8,11,0.073,0.129
minus,23,0,0.209,0.000
other,15,16,0.136,0.188
scope,8,0,0.073,0.000
punctuation,1,1,0.009,0.012
wrong_modality,2,4,0.018,0.047
negation_of_different_term,0,32,0.000,0.376


In [233]:
false_negative_map = {_v:k for k,v in error_types['false_negative'].items() for _v in v}
false_positive_map = {_v:k for k,v in error_types['false_positive'].items() for _v in v}

In [234]:
false_negatives.reset_index(inplace=True, drop=True)
false_negatives.loc[:, 'error_type'] = "UNDEFINED"
false_negatives.loc[:, 'error_type'] = false_negatives.index.map(false_negative_map)

false_positives.reset_index(inplace=True, drop=True)
false_positives.loc[:, 'error_type'] = "UNDEFINED"
false_positives.loc[:, 'error_type'] = false_positives.index.map(false_positive_map)

<ipython-input-234-bcffd6121571>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  false_negatives.loc[:, 'error_type'] = "UNDEFINED"
<ipython-input-234-bcffd6121571>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  false_negatives.loc[:, 'error_type'] = false_negatives.index.map(false_negative_map)
<ipython-input-234-bcffd6121571>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

In [235]:
args = [dcc_dir, results]
kwargs = {'print_html': False, 'print_text': False, 'obfuscate_entity': True}

false_positives['pattern'] = false_positives['entity_id'].apply(lambda x: get_document_text(x, *args, **kwargs)[0])
false_negatives['pattern'] = false_negatives['entity_id'].apply(lambda x: get_document_text(x, *args, **kwargs)[0])

false_positives[['entity_id', 'error_type', 'pattern']].to_csv("results/false-positives_robbert.csv", index=False)
false_negatives[['entity_id', 'error_type', 'pattern']].to_csv("results/false-negatives_robbert.csv", index=False)

<ipython-input-235-92a74258d773>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  false_positives['pattern'] = false_positives['entity_id'].apply(lambda x: get_document_text(x, *args, **kwargs)[0])
<ipython-input-235-92a74258d773>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  false_negatives['pattern'] = false_negatives['entity_id'].apply(lambda x: get_document_text(x, *args, **kwargs)[0])


# biLSTM

## False negatives biLSTM

In [17]:
# Select false negatives
false_negatives = results[(results.label == 'negated') & (results.bilstm == 'not negated')]
false_negatives.head()

,entity_id,category,label,bilstm,bilstm_cv,rule_based,robbert_512_2,robbert_128_2,robbert_32_2,robbert_32_1
2950,GP1191_121_127,GP,negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated
3612,GP1558_64_70,GP,negated,not negated,negated,not negated,not negated,NaN,not negated,NaN
3659,GP1601_67_74,GP,negated,not negated,not negated,not negated,not negated,not negated,not negated,not negated
4432,GP2067_0_5,GP,negated,not negated,not negated,not negated,NaN,NaN,NaN,NaN
5620,GP2796_56_63,GP,negated,not negated,not negated,not negated,not negated,not negated,not negated,negated


In [18]:
# Show issue with a random record
random_entity = false_negatives.sample(1).entity_id.tolist()[0]
text, start, end = get_document_text(random_entity, dcc_dir, results)

Entity: Hepatitis B (515-526)

            entity_id category    label       bilstm    bilstm_cv  \
10513  SP1350_515_526       SP  negated  not negated  not negated   

        rule_based robbert_512_2 robbert_128_2 robbert_32_2 robbert_32_1  
10513  not negated   not negated   not negated          NaN          NaN  


In [19]:
# Count number of false negatives caused by -
count = 0
for index, record in false_negatives.iterrows():
    text, start, end = get_document_text(record.entity_id, dcc_dir, Path('data'), print_text=False)
    if text[end:end+1] == '-':
        count += 1
print(f'{count} of {false_negatives.shape[0]} ({round((count / false_negatives.shape[0]) * 100)}%) false negatives caused by negation described as "-"')

0 of 17 (0%) false negatives caused by negation described as "-"


In [20]:
# Show text for all errors in BiLSTM cross-validation
model_pred = pd.read_csv(bilstm_result_file)

# Load annotated data
with open(annotation_file) as f:
    annotations = json.load(f)
result = []
for document in annotations['projects'][0]['documents']:
    document_name = document['name']
    text = document['text']

    for annotation in document['annotations']:

        # Extract data
        start_char = annotation['start']
        end_char = annotation['end']
        negation_value = annotation['meta_anns']['Negation']['value']

        # Create custom ID
        entity_id = f'{document_name}_{start_char}_{end_char}'
        result.append([entity_id, negation_value])
ann_labels = pd.DataFrame(result, columns=['entity_id', 'label'])  
cmp_labels = pd.merge(left=ann_labels, right = model_pred, left_on='entity_id', right_on='entity_id')
all_errors = cmp_labels[cmp_labels.label != cmp_labels.bilstm_cv]

for error_id, series in all_errors.iterrows():
    get_document_text(cmp_labels["entity_id"].iloc[error_id], dcc_dir, cmp_labels)

Entity: gescheiden (80-90)

        entity_id        label bilstm_cv
117  DL1154_80_90  not negated   negated


Entity: dissociatie (161-172)

          entity_id    label    bilstm_cv
121  DL1156_161_172  negated  not negated


Entity: trauma (76-82)

        entity_id    label    bilstm_cv
155  DL1167_76_82  negated  not negated


Entity: diagnose (340-348)

          entity_id    label    bilstm_cv
360  DL1237_340_348  negated  not negated


Entity: compartimentsyndroom (307-327)

          entity_id    label    bilstm_cv
376  DL1242_307_327  negated  not negated


...

Entity: mictiestoornissen (348-365)

            entity_id        label bilstm_cv
12268  SP2014_348_365  not negated   negated


Entity: tumor (124-129)

            entity_id    label    bilstm_cv
12397  SP2069_124_129  negated  not negated


Entity: claudicatio (40-51)

          entity_id        label bilstm_cv
12435  SP2080_40_51  not negated   negated


Entity: gewichtsverlies (301-316)

            entity_id    label    bilstm_cv
12465  SP2093_301_316  negated  not negated


Entity: ontsteking (22-32)

          entity_id    label    bilstm_cv
12521  SP2113_22_32  negated  not negated


# Rule based

## False negatives


In [21]:
FNs_rule = results[(results.label == 'negated') & (results.rule_based == 'not negated')]['entity_id']
len(FNs_rule)

191

In [22]:
g_FNs_rule = iter(FNs_rule.to_list())

In [23]:
# iterate through one doc at a time
get_document_text(next(g_FNs_rule), dcc_dir, results);

Entity: dissociatie (161-172)

          entity_id category    label   bilstm    bilstm_cv   rule_based  \
121  DL1156_161_172       DL  negated  negated  not negated  not negated   

    robbert_512_2 robbert_128_2 robbert_32_2 robbert_32_1  
121           NaN           NaN          NaN          NaN  


### Observations:
1. Most frequent false negatives are a "list of negations", e.g.:
    - "`ENT`-" or "`ENT` -" or "`ENT`:-" (58 cases)
    - "`ENT`: nee" or "`ENT`: geen" or "`ENT`: negatief" (7 cases)
2. 2nd most frequent are actually not false negatives, but labeling errors (22 cases)
    - negation is labelled as part of the entity, e.g. "geen bijwerking" is the entity (GP1665_79_94, GP1681_0_11, GP2567_126_137)
    - no negation actually present (GP1558_64_70, GP2796_56_63, GP2967_0_6, RD1951_465_472, SP1164_179_186)
    - entity is a (sub)heading of report; not actually present (e.g. SP1188_26_31; only occurs in SP)
3. A negation trigger is missing that could easily be added
    - "neg" / "negatief" (13 cases)
    - "pleit tegen" (4 cases)
    - "niet voorafgegaan" (8 cases)
    - words like "niet" en "geen" that occur directly next to `ENT`, so have a scope of 1; probably too many false positives with broader scope (12 cases)

### Categories

In [24]:
# Load entity_id, snippet containing error, error category, and (absence of) trigger involved
errors_FN = pd.read_csv(result_dir / 'false-negatives_rule-based.csv', sep = ';')
errors_FN.head()

,entity_id,pattern,category_rule_based,category,trigger
0,DL1156_161_172,ENT zijn negatief,missing variation,uncommon negation,negatief
1,DL1163_0_8,ENT treedt niet op,missing trigger,uncommon negation,treden niet op
2,DL1167_76_82,ENT kan niet worden herinnerd,negation of different term,negation of different term,NaN
3,DL1201_0_8,ENT treedt niet op,missing trigger,uncommon negation,treden niet op
4,DL1204_26_34,is er nooit een ENT,missing variation,uncommon negation,nooit


In [30]:
errors_FN[errors_FN.category_rule_based=='wrong modality']

,entity_id,pattern,category_rule_based,category,trigger
32,GP1191_121_127,ENT is over,wrong modality,wrong modality,NaN
109,RD1277_53_64,ENT geresorbeerd,wrong modality,wrong modality,NaN
112,RD1500_47_56,Genezen ENT,wrong modality,wrong modality,NaN
160,SP1483_46_55,Stoppen - ENT,wrong modality,wrong modality,NaN
166,SP1522_199_210,Onbelaste voorgeschiedenis voor ENT,wrong modality,wrong modality,NaN


In [25]:
def print_category_counts(df, err_type):
    cnt = df['category'].value_counts()
    tot = df.shape[0]
    print(f'{err_type} ({tot} total)')
    print(pd.concat([cnt.rename('count'),
               (cnt / tot * 100).rename('percentage')],
              axis=1))

In [26]:
print_category_counts(errors_FN, 'False negatives')

False negatives (191 total)
                            count  percentage
uncommon negation              62   32.460733
minus                          51   26.701571
other                          29   15.183246
annotation error               20   10.471204
punctuation                    13    6.806283
speculation                     8    4.188482
wrong modality                  6    3.141361
negation of different term      2    1.047120


In [27]:
def print_example_error(df, category):
    smpl = df[df['category'] == category].sample()
    get_document_text(smpl.entity_id.values[0], dcc_dir, results.iloc[:,0:3]);
    print(smpl)

In [28]:
print_example_error(errors_FN, 'sentence splitting')

ValueError: a must be greater than 0 unless no samples are taken

## False positives


In [ ]:
FPs_rule = results[(results.label == 'not negated') & (results.rule_based == 'negated')]['entity_id']
len(FPs_rule)

In [ ]:
g_FPs_rule = iter(FPs_rule.to_list())

In [ ]:
# iterate through one doc at a time
get_document_text(next(g_FPs_rule), dcc_dir, results);

### Observations:
- "wel" should be a termination trigger
- triggers like "geen" and "niet" should have a reduced scope (maybe even just 1?)
    - might also help to add punctuation like `,` and `;` as termination triggers

### Categories

In [ ]:
# Load entity_id, snippet containing error, error category, and (absence of) trigger involved
errors_FP = pd.read_csv(result_dir / 'false-positives_rule-based.csv', sep = ';')
errors_FP.head()

In [ ]:
print_category_counts(errors_FP, 'False positives')

In [ ]:
print_example_error(errors_FP, 'missing termination trigger')

# Ensemble

In [ ]:
results[['label', 'bilstm', 'bilstm_cv', 'rule_based', 
         'robbert_512_2', 'robbert_128_2', 'robbert_32_2']]=\
results[['label', 'bilstm', 'bilstm_cv', 'rule_based', 
         'robbert_512_2', 'robbert_128_2', 'robbert_32_2']].apply(lambda x: x.map({'not negated':0,
                                                                                   'negated': 1}), 
                                                                  axis=1)
results.dropna(subset=['robbert_512_2'], inplace=True)
results.drop(['bilstm', 'robbert_128_2', 'robbert_32_2'], axis=1, inplace=True)

In [ ]:
'''
ACC = (TP+TN)/(TP+TN+FP+FN)
SPEC = TN/(TN+FP)
SENS = TP/(TP+FN)
PPV = TP/(TP+FP)
NPV = TN/(TN+FN)
'''

results['ensemble_mv'] = results[['bilstm_cv', 'rule_based', 'robbert_512_2']].\
                            apply(lambda x: round(np.mean(x)),axis=1)

In [ ]:
def confusion_matrix(ys,est='ensemble_mv'):
    y_est = ys[est]
    y_true = ys['label']
    TP = sum((y_est==y_true) & (y_true==1))
    TN = sum((y_est==y_true) & (y_true==0))
    FP = sum((y_est!=y_true) & (y_true==0))
    FN = sum((y_est!=y_true) & (y_true==1))
    
    ACC = (TP+TN)/(TP+TN+FP+FN)
    SPEC = TN/(TN+FP)
    SENS = TP/(TP+FN)
    PPV = TP/(TP+FP)
    NPV = TN/(TN+FN)

    return round(ACC,3),round(SPEC,3),round(SENS,3),round(PPV,3),round(NPV,3)

results[['category', 'label', 'ensemble_mv']].groupby('category').apply(confusion_matrix)

In [ ]:
results[['category', 'label', 'bilstm_cv']].groupby('category').apply(lambda x: confusion_matrix(x,est='bilstm_cv'))

In [ ]:
results[['category', 'label', 'rule_based']].groupby('category').apply(lambda x: confusion_matrix(x,est='rule_based'))

In [ ]:
results[['category', 'label', 'robbert_512_2']].groupby('category').apply(lambda x: confusion_matrix(x,est='robbert_512_2'))